In [21]:
from langchain_groq import ChatGroq
from pydantic import BaseModel
from dotenv import load_dotenv
import os
from langchain.tools import tool
from langchain_core.tools import StructuredTool
from typing import ClassVar, Type
from pydantic import BaseModel, Field
from langchain.tools import StructuredTool 

In [22]:
load_dotenv()
# groq_api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(model="llama-3.3-70b-versatile")

In [23]:
llm.invoke("hi")

AIMessage(content='Hello. How can I help you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 36, 'total_tokens': 46, 'completion_time': 0.027290406, 'prompt_time': 0.001560397, 'queue_time': 0.055011723, 'total_time': 0.028850803}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_34d416ee39', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--917ae0d1-8cb4-4299-8ed8-a28d46914dec-0', usage_metadata={'input_tokens': 36, 'output_tokens': 10, 'total_tokens': 46})

In [24]:
@tool
def multiply(a: int, b: int) -> int:
    """
    Multiply two integers.

    Args:
        a (int): The first integer.
        b (int): The second integer.

    Returns:
        int: The product of a and b.
    """
    return a * b


In [25]:
multiply




StructuredTool(name='multiply', description='Multiply two integers.\n\nArgs:\n    a (int): The first integer.\n    b (int): The second integer.\n\nReturns:\n    int: The product of a and b.', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x0000027AFAE02A70>)

In [26]:
class WeatherInput(BaseModel):
    city: str


In [27]:
def get_weather(city: str) -> str:
    """
    Get the weather for a given city.

    Args:
        city (str): The name of the city.

    Returns:
        str: A string describing the weather in the city.
    """
    return f"The weather in {city} is sunny."

In [28]:
weather_tool = StructuredTool.from_function(
    func=get_weather,
    name="get_weather",
    description="Fetches real-time weather data for a city",
    args_schema=WeatherInput,  
)

In [29]:
class WeatherInput(BaseModel):
    city: str = Field(..., description="City name")
    units: str = Field("metric", description="metric or imperial")

class GetWeatherTool(StructuredTool):
    name: ClassVar[str] = "get_weather"           
    description: ClassVar[str] = "Fetches weather data for a city"
    args_schema: ClassVar[Type[BaseModel]] = WeatherInput

    def _run(self, city: str, units: str = "metric") -> str:
        return get_weather(city, units)  